<a href="https://colab.research.google.com/github/smf-9000/learning-and-practicing/blob/main/1_dynamic_programming_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First impression:** DP seems very interesting, and I also have a strange impression that practice is more than necessary for this field. It, in some weird way, resemble me to Factorization of polynomials. Better to not explain :)

I will start with finding solutions for problems from the DP field and try later to find a connection with theory.

# Code / Problems

## 10. Regular Expression Matching

10. Regular Expression Matching (`https://leetcode.com/problems/regular-expression-matching/`)

Given an input string `s` and a pattern `p`, implement regular expression matching with support for `'.'` and `'*'` where:

`'.'` Matches any single character.​​​​
`'*'` Matches zero or more of the preceding element.

The matching should cover the entire input string (not partial).



---



Usually, in this situation, we have some DP matrix which we have to fill in a clever way. So we have to create a few rules for filling the DP matrix.

Let's associate the x-axis to the pattern and the y-axis to the string. The idea is to test the pattern matching to the string by incrementally increasing both of them. To clarify, at the start, both the pattern and the string will be zero-length, and we will be testing them one to another until both of them reach full length.

The column on the most right will tell us whether the whole pattern matches one of the states `[the empty string, the first character of the string, the first two characters of the string, ..., the whole string]`.

Steps:

It is obvious that `DP[0][0] = True`

If we encounter a letter in the pattern, we have situation: 

> `True if DP[i-1][j-1] == True and s[i] = p[j] else False`

if we encounter a dot symbol (".") in the pattern, we have: 

> `True if DP[i-1][j-1] == True else False`

and if we encounter a asterisk symbol ("*") in the pattern, we have:

> `True for DP[i][j-1] == True`

> `True for DP[i-1][j] == True and char_before == '.'`

> `True for DP[i-1][j] == True and char_before == s[i]`

The last cell in the last row of the matrix contains the answer to the question "does pattern match string?" the result.

> `result = DP[-1][-1]`


In [ ]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        A = [c for i, c in enumerate(list(p)) if i < len(p)-1 and p[i+1] == '*']
        p = re.sub(r'.\*', '*', p)
        D = [[False] * (len(p)+1) for _ in range(len(s)+1)]   # +1 for 0th step
        p_p = None
        for r in range(len(D)):
            D[r][0] = True if r == 0 else False
            k = 0
            for c in range(1, len(D[0])):
                i = r - 1
                j = c - 1
                asterisk = True if p[j] == '*' else False
                p_c = p[j]
                if asterisk:
                    p_c = A[k]
                    k += 1
                if r == 0:
                    if p[j] == '*':
                        D[r][c] = D[0][c-1]
                else:
                    if p_c == '.':
                        if asterisk:
                            D[r][c] = D[r][c-1] or D[r-1][c]
                        else:
                            D[r][c] = D[r-1][c-1]
                    else:
                        if asterisk:
                            D[r][c] = D[r][c-1] or (D[r-1][c] and s[i] == p_c)
                        else:
                            D[r][c] = D[r-1][c-1] and s[i] == p_c
                p_p = p_c
        return D[-1][-1]



---
---
---



## 312. Burst Balloons

312. Burst Balloons `https://leetcode.com/problems/burst-balloons/`

You are given `n` balloons, indexed from `0` to `n - 1`. Each balloon is painted with a number on it represented by an array `nums`. You are asked to burst all the balloons.

If you burst the `ith` balloon, you will get `nums[i - 1] * nums[i] * nums[i + 1]` coins. If `i - 1` or `i + 1` goes out of bounds of the array, then treat it as if there is a balloon with a `1` painted on it.

Return the `maximum coins you can collect by bursting the balloons wisely`.



---
classical DP: ⬇


If we try on the first step to divide this problem into the left and right parts, we can recognize that is not possible. For the top-down approach, we need all balloons present in one list. But, if we try to split the problem at the last step on the left and right parts, that seems achievable.
```
As we have added '1' on either side of the list, our last step would be similar to this one:
[1.......x.............1]
 =======================
1*x*1 + left_region + right_region

The step before the last would be similar to this one:
[1.......x......y......1]
         ===============
x*y*1 + (x,y)region + (y,end)region

```
For this pseudocode, we don't need an additional explanation. If we imagine these steps to the end, we recognize that we need solutions for all adjacent regions from length 3 to length len(N)-1 if we want to cover all combinations for the potential final step.

Considering how we write down data in the DP matrix, it is clear that the final solution is `D[0][size-1]`.

In [ ]:
class Solution:
    def maxCoins(self, nums: List[int]) -> int:
        N = [e for e in [1, *nums, 1] if e > 0]
        size = len(N)
        D = [[0] * size for _ in range(size)]
        for d in range(2, size):
            for l in range(0, size-d):
                r = l + d
                for m in range(l+1, r):
                    D[l][r] = max(D[l][r], N[l] * N[m] * N[r] + D[l][m] + D[m][r])
        return D[0][size-1]



---

something relatively new: ⬇

As we code in python, there is one nice functionality to tight a code a bit. `@functools.cache(user_function)`

***New in version 3.9.*** `https://docs.python.org/3/library/functools.html`

> `def: Simple lightweight unbounded function cache. Sometimes called “memoize”.`

This is practically a replacement for the DP matrix. We just need to create a function with no objects as arguments and "decorate" it.

Follows refactored code for this new functionality:

In [ ]:
from functools import cache

class Solution:
    def maxCoins(self, nums: List[int]) -> int:
        N = [e for e in [1, *nums, 1] if e > 0]

        @cache
        def rec(a, b):
            n = 0
            for i in range(a+1, b):
                t = N[a]*N[i]*N[b] + rec(a, i) + rec(i, b)
                if n < t:
                    n = t
            return n
        
        return rec(0, len(N)-1)

 😊



---
---
---



## 87. Scramble String

87. Scramble String `https://leetcode.com/problems/scramble-string/`

We can scramble a string `s` to get a string `t` using the following algorithm:

1. If the length of the string is 1, stop.
2. If the length of the string is > 1, do the following:
> * Split the string into two non-empty substrings at a random index, i.e., if the string is `s`, divide it to `x` and `y` where `s = x + y`.
> * **Randomly** decide to swap the two substrings or to keep them in the same order. i.e., after this step, `s` may become `s = x + y` or `s = y + x`.
> * Apply step 1 recursively on each of the two substrings `x` and `y`.

Given two strings `s1` and `s2` of **the same length**, return `true` if `s2` is a scrambled string of `s1`, otherwise, return `false`.



---



After breaking a string into two parts, left and right, we can find each in the original (side-by-side). We need to catch this understanding, no matter how simple it was, and apply it to further steps because every next recursive step deals with the unmodified part of the original.

Further, after breaking one of the parts into two new parts, we can find each of the new parts in the original string from the beginning. Those two new parts will be side-by-side in the original.

One step before the end, we have a small part of two letters, and we split it into two parts of one letter each. Again, as a consequence of recursion, we can find these two letters in the original string side-by-side.

When I say "side-by-side", I mean in both directions.

It seems to be pretty basic reasoning, but this is the solution to the problem, just this much.

Let's write code for this.

In [ ]:
class Solution:
    def isScramble(self, s1: str, s2: str) -> bool:
        if s1 == s2:
            return True
        l = len(s1)
        D = [[[False] * l for _ in range(l)] for _ in range(l)]
        
        for d in range(l):
            for i1 in range(0, l-d):
                for i2 in range(0, l-d):
                    if d == 0:
                        D[i1][i2][0] = s1[i1] == s2[i2]
                    else:
                        for m in range(0, d):
                            if not D[i1][i2][d]:
                                D[i1][i2][d] = (D[i1][i2][m] and D[i1+m+1][i2+m+1][d-m-1]) or (D[i1][i2+m+1][d-m-1] and D[i1+d-m][i2][m]) # # # #
        return D[0][0][l-1]
        



```
# d == 0 -> one letter string
# d == 1 -> two letter string
# ...

#     |<--m--->|
# s1: 1--------x------------2
# s2: 1--------x------------2
# D[i1][i2][m] means s1[i1:i1+m] ~~ s2[i2:i2+m]

#              |<----d-m--->|
# s1: 1--------x------------2
# s2: 1--------x------------2
# D[i1+m+1][i2+m+1][d-m-1] means s1[i1+m+1:i1+d] ~~ s2[i2+m+1:i2+d]

#     |<--m--->|   |<--m--->|
# s1: 1------------x--------2
# s2: 1--------x------------2
#     |<----d-m--->|
#              |<----d-m--->|
# D[i1][i2+m+1][d-m-1] means s1[i1:i1+d-m] ~~ s2[i2+m+1:i2+d]

#     |<--m--->|   |<--m--->|
# s1: 1------------x--------2
# s2: 1--------x------------2
# D[i1+d-m][i2][m] means s1[i1+d-m:i1+d] ~~ s2[i2:i2+m]
```





---
---
---



## 115. Distinct Subsequences 

115. Distinct Subsequences `https://leetcode.com/problems/distinct-subsequences/`

Given two strings `s` and `t`, *return the number of distinct subsequences of `s` which equals `t`*.

A string's **subsequence** is a new string formed from the original string by deleting some (can be none) of the characters without disturbing the remaining characters' relative positions. (i.e., `"ACE"` is a subsequence of `"ABCDE"` while `"AEC"` is not).

The test cases are generated so that the answer fits on a 32-bit signed integer.

In [ ]:
class Solution:
    def numDistinct(self, s: str, t: str) -> int:
        l_s = len(s)
        l_t = len(t)
        D = [[0] * l_t for _ in range(l_s)]
        for c in range(l_t):
            for r in range(l_s):
                if t[c] == s[r]:
                    if r < c:
                        D[r][c] = 0
                    elif c == 0:
                        D[r][c] = D[r-1][c] + 1
                    else:
                        D[r][c] = D[r-1][c-1] + D[r-1][c]
                else:
                    if r >= c:
                        D[r][c] = D[r-1][c]
        return D[-1][-1]

`Just this much.`

---
---
---

## 140. Word Break II

140. Word Break II `https://leetcode.com/problems/word-break-ii/`

Given a string `s` and a dictionary of strings `wordDict`, add spaces in `s` to construct a sentence where each word is a valid dictionary word. Return all such possible sentences in **any order**.

**Note** that the same word in the dictionary may be reused multiple times in the segmentation.



In [ ]:
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        l = len(s)
        M = {e: len(e) for e in wordDict}
        D = []
        
        for i in range(1, l+1):
            D.append([])
            for k, v in M.items():
                if i >= v:
                    if k == s[i-v:i]:
                        if i-v == 0:
                            D[i-1].append(k)
                        else:
                            for b in D[i-v-1]:
                                D[i-1].append(b + " " + k)
        return D[-1]
                            
# Reasoning:                       
                            
# |***************|
# |---|               ok  ---
#  |***|              x -> there is no previous part that matches its start
# |-----|             ok  ---
#     |-----|         ok  --- -----
#   |-------|         x -> there is no previous part that matches its start
#       |---|         ok  ----- ---
#           |----|    ok [--- ----- ----, ----- --- ----]
#       |--------|    ok  ----- --------
#                |-|  ok [--- ----- ---- -, ----- --- ---- -, ----- -------- -]

---
---
---

## 124. Binary Tree Maximum Path Sum

124. Binary Tree Maximum Path Sum `https://leetcode.com/problems/binary-tree-maximum-path-sum/`


A **path** in a binary tree is a sequence of nodes where each pair of adjacent nodes in the sequence has an edge connecting them. A node can only appear in the sequence **at most once**. Note that the path does not need to pass through the root.

The **path sum** of a path is the sum of the node's values in the path.

Given the `root` of a binary tree, return *the maximum **path sum** of any **non-empty** path*.

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def maxPathSum(self, root: TreeNode) -> int:
        _m = -1000
        
        def rec(c):
            nonlocal _m
            curr = None
            send = None
            if c.left is None and c.right is None:
                curr = c.val
                send = c.val
            elif c.right is None:
                curr = send = max(c.val, c.val + rec(c.left))
            elif c.left is None:
                curr = send = max(c.val, c.val + rec(c.right))
            else:
                l = rec(c.left)
                r = rec(c.right)
                curr = max(c.val, c.val + l, c.val + r, c.val + l + r)
                send = max(c.val, c.val + l, c.val + r)
            if curr > _m:
                _m = curr
            return send
        
        rec(root)
        return _m

---
---
---

## 354. Russian Doll Envelopes

354. Russian Doll Envelopes `https://leetcode.com/problems/russian-doll-envelopes/`

You are given a 2D array of integers envelopes where `envelopes[i] = [wi, hi]` represents the width and the height of an envelope.

One envelope can fit into another if and only if both the width and height of one envelope are greater than the other envelope's width and height.

Return *the maximum number of envelopes you can Russian doll (i.e., put one inside the other)*.

**Note:** You cannot rotate an envelope.

In [ ]:
class Solution:
    def maxEnvelopes(self, envelopes: List[List[int]]) -> int:
        E = envelopes
        E.sort(key=lambda e: (e[0], -e[1]))
        D = [E[0][1]]

        for [w, h] in E:
            if h > D[-1]:
                D.append(h)
            else:
                for i in range(len(D)-1, -1, -1):
                    if i == 0 or (D[i] >= h and D[i-1] < h):
                        D[i] = h
                        break
        return len(D)

```
Value in position i-1 in D array shows how high the envelope should be to digest i-1 smaller envelopes. 
Steps:
- Suppose we encounter an envelope with a higher height than the last value in D. 
- - in that case, we can ADD its height in D. 
- If we meet an envelope that has a smaller height than the last value in D, 
- - we check if it is higher than the value before the last value in D. 
- - - If this is so, we can replace the last value in D with its height; 
- - - if not, we decrement checking positions and try again, etc.

D=[]                   E=[[46, 89], [50, 53], [52, 68], [72, 45], [77, 81]]
D=[89]                 E=[[50, 53], [52, 68], [72, 45], [77, 81]]
D=[53]                 E=[[52, 68], [72, 45], [77, 81]]
D=[53,68]              E=[[72, 45], [77, 81]]
D=[45,68]              E=[[77, 81]]
D=[45,68,81]           E=[]
```

---
---
---

# `Theory`

## ...